In [1]:
import os

In [2]:
pwd

'd:\\Oslo\\OsloMet\\Fourth semester\\Electoral_Symbols_And_Vote_Detection_MLOPS\\Electoral_Symbols_And_Vote_Detection\\research\\faster-rcnn'

In [3]:
os.chdir("../../")

In [4]:
%pwd

'd:\\Oslo\\OsloMet\\Fourth semester\\Electoral_Symbols_And_Vote_Detection_MLOPS\\Electoral_Symbols_And_Vote_Detection'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weight: str
    params_classes: int


In [6]:
from src.modules.symbol_detection.faster_rcnn.constants import *
from src.utils.common import read_yaml, create_directories

In [9]:
class ConfigurationManager: 
    def __init__(
                self,
                config_filepath =  CONFIG_FILE_PATH,
                params_filepath = PARAMS_FILE_PATH
        ):
            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)

            create_directories([self.config.artifacts_root])
    
    def get_prepare_base_model_config(self)-> PrepareBaseModelConfig:
          config = self.config.prepare_base_model

          create_directories([config.root_dir])

          prepare_base_model_config = PrepareBaseModelConfig(
                root_dir= Path(config.root_dir),
                base_model_path=Path(config.base_model_path),
                updated_base_model=Path(config.updated_base_model),
                params_image_size=self.params.IMAGE_SIZE,
                params_learning_rate=self.params.LEARNING_RAGE,
                params_include_top=self.params.INCLUDE_TOP,
                params_weight=self.params.WEIGHTs,
                params_classes=self.params.CLASSES
                )
          return prepare_base_model_config

In [10]:
import os
import urllib.request as request
from zipfile import ZipFile
import torchvision 
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

In [12]:
class PrepareBaseModel:
    def __init__(self,config) -> PrepareBaseModelConfig:
        self.config = config
    
    def get_faster_rcnn_model(self):

        self.model = torchvision.models.detection.fasterrcnn_resnet50_fpn_v2(pretrained=True)    
        # get number of input features for the classifier
        in_features = self.model.roi_heads.box_predictor.cls_score.in_features
        # replace the pre-trained head with a new one
        self.model.roi_heads.box_predictor = FastRCNNPredictor(in_features, self.config.params_classes) 
        return self.model